In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def tmp_parse(message, separator='\x01', valueSeparator='='):
    head = {}
    tmp_ind = message.find('268=')
    if tmp_ind == -1:
        #print(message)
        return None
    # assert tmp_ind != -1, "TAG 268 not found!"
    split = message.find(separator, tmp_ind)
    n = 0
    while n < split:
        # Parse the next tag
        nextValueSeparator = message.index(valueSeparator, n)
        tag = message[n:nextValueSeparator]
        # Parse the next value
        nextSeparator = message.index(separator, nextValueSeparator)
        value = message[nextValueSeparator + 1:nextSeparator]
        n = nextSeparator + 1
        assert head.get(tag) is None, "Illegal duplicated values!"
        head[tag] = value
    return head


def test_message(message, separator='\x01', valueSeparator='='):
    if len(message) == 0 or message[-1] != separator:
        raise ValueError('FIX Message is invalid!')
    a_ind = message.find('1023=')
    b_ind = message.find('37705=')
    c_ind = message.find('269=2')
    return a_ind, b_ind, c_ind


In [ ]:
year = '2022'
month = '01'
day = '03'
datestr = year + month + day
directory = 'C:/data/CME/2022/{0}/{1}/MBO/XCME/'.format(month, day)
name = '{0}-MBO_xcme_es_fut_eth.gz'.format(datestr)
file = directory + name

f = gzip.open(file, 'rb')
i = 0
# my_set = set({})
for line in f:
    message = line.decode('utf-8')[:-1]
    flag = False
    if i % 500000 == 0:
        print(i)
    i += 1
#     if message.find('55=') == -1:
#         print(i)
#         print(message)
#         print()
#     a_ind, b_ind, c_ind = test_message(message)
    head = tmp_parse(message)
    if head is None:
        continue
    if head['5799'][-3:] == '000':
        print(i)
        print(message)
        print()
#     if c_ind == -1 and a_ind == -1 and b_ind != -1:
#         print(i)
#         print(message)
#         print()
    # head, block_list = parseMessage(line.decode('utf-8')[:-1])
#     head = tmp_parse(line.decode('utf-8')[:-1])
#     if head is None:
#         continue
#     if head['5799'] == '10000100':
#         print(line.decode('utf-8')[:-1])
#         # print(head)
#         print()
    # my_set.add(head['5799'])

In [ ]:
year = '2022'
month = '01'
day = '03'
datestr = year + month + day
directory = 'C:/data/CME/2022/{0}/{1}/MBO/XCME/'.format(month, day)
name = '{0}-MBO_xcme_es_fut_eth.gz'.format(datestr)
file = directory + name

f = gzip.open(file, 'rb')
i = 0
for line in f:
    message = line.decode('utf-8')[:-1]
    flag = False
    if i % 500000 == 0:
        print(i)
    i += 1
    # head, block_list = parseMessage(message)
    # head = tmp_parse(message)
    head, block_list = parseOrderMessage(message)
    print(block_list)
    if head is None:
        continue

In [2]:
import gzip
from datetime import datetime
from CMEdataUtil import MarketDepth, parseOrderMessage, OrderEvent, FillEvent, TradeEvent, update_levels

def block_to_order_indi1(block, head):
    return OrderEvent(head['60'], block['37'], block['55'], block['279'], block['269'], block['270'], 
                      block['37706'])

def block_to_order_indi4(block, level_block, head):
    return OrderEvent(head['60'], block['37'], level_block['55'], block['37708'], level_block['269'], 
                      level_block['270'], block['37706'])

def block_to_trade(block, head, agg_id, agg_qty):
    return TradeEvent(head['60'], block['55'], 'trade', block['5797'], block['270'], block['271'], agg_id, agg_qty, block['346'])
    
def write_orders(order_list, f):
    for o in order_list:
        f.write(str(o))

orders_head = 'TRANSACTTIME,PUBLIC_ORDER_ID,SYMBOL,EXECTYPE,SIDE,PRICE,VISIBLEQTY\n'
trades_head = 'TRANSACTTIME,SYMBOL,EXECTYPE,SIDE,EXECUTEDPRICE,EXECUTEDSIZE,PUBLICORDERID,PUBLICORDERQTY,ORDERSNUM\n'
fills_head = 'TRANSACTTIME,SYMBOL,PUBLICORDERID,PUBLICORDERQTY\n'
levels_head = "time,"+"".join(["BidPrc_%d,BidQty_%d,BidNum_%d,"%(i,i,i) for i in range(1, 11)]) + "".join(["AskPrc_%d,AskQty_%d,AskNum_%d,"%(i,i,i) for i in range(1, 10)]) + "AskPrc_10,AskQty_10,AskNum_10\n"
print(len(levels_head.split(',')))

ticker = 'ESM2'
year = '2022'
month = '01'
day = '02'

datestr = year + month + day
# if datetime.strptime(datestr, '%Y%m%d').weekday() >= 5:
#     print("Skip %s: weekend!"%datestr)
#     continue
print("Start parsing data for %s"%datestr)
directory = 'C:/data/CME/2022/{0}/{1}/MBO/XCME/'.format(month, day)
name = '{0}-MBO_xcme_es_fut_eth.gz'.format(datestr)
file = directory + name


bid_depth = MarketDepth(total_level=10, side='bid')
ask_depth = MarketDepth(total_level=10, side='ask')
fl = open(directory+ticker+'_levels.csv', 'w')
fl.write(levels_head)
ft = open(directory+ticker+'_trades.csv', 'w')
ft.write(trades_head)
fo = open(directory+ticker+'_orders.csv', 'w')
fo.write(orders_head)
ff = open(directory+ticker+'_fills.csv', 'w')
ff.write(fills_head)

f = gzip.open(file, 'rb')
i = 0

for line in f:
    message = line.decode('utf-8')[:-1]
    if i % 500000 == 0:
        print(i)
    i += 1
    head, block_list = parseOrderMessage(message)
    if head is None:
        continue

    if head['000'] == '0':
        # trades
        assert len(block_list) == int(head['268']) + int(head['37705'])
        num_entries = int(head['268'])
        cumsum_id = num_entries
        flag = True # Used for the case when 5797=0 is in the middle of several blocks in one message
        for idx in range(num_entries):
            block = block_list[idx]
            assert block['279'] == '0'
            if cumsum_id >= len(block_list): # For the case when there is some orders missing in trades message
                print(i, 'cumsum_id larger than length of block_list!', cumsum_id, len(block_list))
                break
            if block['55'] == ticker:
                left, right = cumsum_id, cumsum_id + int(block['346'])
                for tmp_b in block_list[left:right]:
                    _fill = FillEvent(head['60'], ticker, tmp_b['37'], tmp_b['32'])
                    ff.write(str(_fill))
                
                if block['5797'] == '0':
                    print(i, ' aggressor tag 5797 is 0 !!!')
                    flag = False
                    cumsum_id += int(block['346'])
                    continue
                assert block_list[cumsum_id]['32'] == block['271']
                t = block_to_trade(block, head, block_list[cumsum_id]['37'], block_list[cumsum_id]['32'])
                ft.write(str(t))
            
            cumsum_id += int(block['346'])
        
        if flag and cumsum_id != num_entries + int(head['37705']):
            print(i, 'cumsum_id and length of block_list mismatch!', cumsum_id, num_entries + int(head['37705']))
    elif head['000'] == '1':
        assert int(head['268']) == len(block_list)
        for block in block_list:
            if block['55'] == ticker and (block['269'] == '0' or block['269'] == '1'):
                o = block_to_order_indi1(block, head)
                fo.write(str(o))
        
    elif head['000'] == '2':
        assert len(block_list) == int(head['268']), "block number mismatch!"
        flag = update_levels(bid_depth, ask_depth, block_list, ticker)
        bid_str, ask_str = str(bid_depth), str(ask_depth)
        if flag and bid_str != "" and ask_str != "":
            assert bid_str[-1] == ask_str[-1] == "\n", "Error in depth string!"
            res_str = head['60'] + "," + bid_str[:-1] + "," + ask_str
            assert len(res_str.split(",")) == 61, "Error when generating result string!"
            fl.write(res_str)
            
    elif head['000'] == '3':
        raise Exception('Unexpected value 3 for 000 in head')
    
    elif head['000'] == '4':
        assert len(block_list) == int(head['268']) + int(head['37705'])
        split = int(head['268'])
        flag = update_levels(bid_depth, ask_depth, block_list[:split], ticker)
        bid_str, ask_str = str(bid_depth), str(ask_depth)
        if flag and bid_str != "" and ask_str != "":
            assert bid_str[-1] == ask_str[-1] == "\n", "Error in depth string!"
            res_str = head['60'] + "," + bid_str[:-1] + "," + ask_str
            assert len(res_str.split(",")) == 61, "Error when generating result string!"
            fl.write(res_str)
            
        for block in block_list[split:]:
            level_block = block_list[int(block['9633'])-1]
            if level_block['269'] != '0' and level_block['269'] != '1':
                raise Exception('Invalid level block!')
            if level_block['55'] == ticker:
                o = block_to_order_indi4(block, level_block, head)
                fo.write(str(o))
        
    else:
        raise Exception('Unknown value for 000 in head')
    
    

f.close()
fl.close()
ft.close()
fo.close()
ff.close()

61
Start parsing data for 20220102
0
1128=99=12735=f49=CME34=23752=2022010211160648774684260=2022010211155649568515175=202201025799=100000001151=ES326=18327=11174=010=146
1128=99=51335=d49=CME75=2022010234=93552=202201021704592821630995799=00000000980=A779=202201021704591647975811180=3101300=6455=ESZ348=31486322=8200=2023121151=ES6937=ES167=FUT461=FFIXSX9779=N462=5207=XCME15=USD1142=F562=11140=3000969=25.01146=12.59787=0.011141=11022=GBX264=10864=2865=51145=20210604-21:30:00.000000000865=71145=20231215-14:30:00.000000000870=1871=24872=00000000000001000010000000001111996=IPNT1147=501149=508625.01148=441925.01143=600.01150=475300.0731=0000001110=194
1128=99=51335=d49=CME75=2022010234=93652=202201021704592823488585799=00000000980=A779=202201021704591647975811180=3101300=6455=ESZ548=29497322=8200=2025121151=ES6937=ES167=FUT461=FFIXSX9779=N462=5207=XCME15=USD1142=F562=11140=3000969=25.01146=12.59787=

In [4]:
f.close()
fl.close()
ft.close()
fo.close()
ff.close()

In [7]:
head, len(block_list)

({'1128': '9',
  '9': '279',
  '35': 'X',
  '49': 'CME',
  '75': '20220103',
  '34': '10802',
  '52': '20220102230007887992105',
  '60': '20220102230007887344271',
  '5799': '00000001',
  '268': '1',
  '000': '0',
  '37705': '4'},
 5)

In [8]:
block_list

[{'279': '0',
  '269': '2',
  '48': '5782',
  '55': 'ESH2',
  '83': '4787',
  '270': '477625.0',
  '271': '3',
  '346': '4',
  '5797': '1',
  '37711': '27329'},
 {'37': '649196488319', '32': '3'},
 {'37': '649196487167', '32': '1'},
 {'37': '649196487172', '32': '1'},
 {'37': '649196487138', '32': '1', '10': '061'}]